In [15]:
import math

# Dictionary of Van der Waals radii in Angstroms for common CHONPS atoms
van_der_waals_radii = {
    'H': 1.2,  # Hydrogen
    'C': 1.7,  # Carbon
    'N': 1.55,  # Nitrogen
    'O': 1.52,  # Oxygen
    'S': 1.8,   # Sulfur
    'P': 1.8    # Phosphorus (in phosphate groups)
}

#### Same as stateAnalysis_tools for eventual functions merge
def parse_pdb(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

def get_atom_coordinates(lines):
    atoms = []
    for line in lines:
        if line.startswith('ATOM') or line.startswith('HETATM'):
            try:
                atom_name = line[12:16].strip()[0]  # First letter represents element (H, C, N, O, S, P, etc.)
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                atoms.append((atom_name, x, y, z))
            except ValueError:
                print(f"Error parsing line: {line}")
    return atoms

def calculate_distance(atom1, atom2):
    _, x1, y1, z1 = atom1
    _, x2, y2, z2 = atom2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
################################

# New stuff:
def check_clashes(atoms, tolerance_percentage=0):
    clashes = []
    tolerance_factor = 1 + (tolerance_percentage / 100.0)  # Adjust Van der Waals radii based on tolerance

    for i, atom1 in enumerate(atoms):
        for j, atom2 in enumerate(atoms):
            if i != j:
                dist = calculate_distance(atom1, atom2)
                # Adjust Van der Waals radii based on tolerance
                atom1_name, atom2_name = atom1[0], atom2[0]
                radius1 = van_der_waals_radii.get(atom1_name, 1.7) * tolerance_factor  # Default to 1.7 for unknown atoms
                radius2 = van_der_waals_radii.get(atom2_name, 1.7) * tolerance_factor

                if dist < radius1 + radius2:
                    clashes.append((atom1, atom2, dist))
    return clashes



In [16]:
# Example usage with a tolerance of -10% (less strict) and +10% (more strict)
file_path_1 = '/Users/adrianahernandezgonzalez/LabNotebook/10-24/states/partialAlphaCaV12HS8HLPlocalrun_b3702_8_16_10/pdb/model_1_ptm_r0_seed2.pdb'
file_path_2 = '/Users/adrianahernandezgonzalez/LabNotebook/10-24/states/partialAlphaCaV12HS8HLPlocalrun_b3702_256_512_10/pdb/model_5_ptm_r12_seed0.pdb'

tolerance = -50  # Adjust this value for stricter or looser clash detection (-10 for looser, +10 for stricter)
lines_1 = parse_pdb(file_path_1)
lines_2 = parse_pdb(file_path_2)

atoms_1 = get_atom_coordinates(lines_1)
atoms_2 = get_atom_coordinates(lines_2)

clashes_1 = check_clashes(atoms_1, tolerance_percentage=tolerance)
clashes_2 = check_clashes(atoms_2, tolerance_percentage=tolerance)

# Report clashes for each model
print(f"Model 1 (file 1) clashes with {tolerance}% tolerance: {len(clashes_1)}")
print(f"Model 2 (file 2) clashes with {tolerance}% tolerance: {len(clashes_2)}")


Model 1 (file 1) clashes with -50% tolerance: 27562
Model 2 (file 2) clashes with -50% tolerance: 24410
